# 대화 상태 (Conversation State)
OpenAI는 대화 상태를 관리하는 몇 가지 방법을 제공하며, 이는 대화에서 여러 메시지 또는 차례에 걸쳐 정보를 보존하는 데 중요합니다.

In [1]:
from dotenv import load_dotenv
load_dotenv() 

True

In [2]:
from openai import OpenAI

client = OpenAI()

Model = "gpt-5-nano"

### 수동으로 대화 상태 관리하기 

각 텍스트 생성 요청은 독립적이고 상태가 없는(stateless) 요청이지만, 여러 턴에 걸친 대화를 구현하기 위해서는 이전 메시지들을 함께 전달할 수 있습니다.  

이전 메시지 기록을 매번 요청에 포함시켜 전달하면,
모델은 이전 문맥을 기억하며 대화를 이어갈 수 있습니다.

In [3]:
response = client.responses.create(
    model=Model,
    input=[
        {"role": "user", "content": "똑똑똑"},
        {"role": "assistant", "content": "누구세요?"},
        {"role": "user", "content": "엄마예요."},
    ],
)

print(response.output_text)

엄마 반가워요! 어서 들어오세요. 어디 앉으실래요? 차 한잔 드릴까요, 아니면 오늘 필요한 게 있나요? 제가 바로 도와드릴게요.


### 대화 상태용 OpenAI API
response API를 사용하면 대화 상태를 자동으로 관리할 수 있습니다.  
Responses API는 상태를 저장하므로 대화 전반의 컨텍스트 관리가 간단한 매개변수로 가능합니다.

In [4]:
# OpenAI Responses API 예제: previous_response_id 로 대화 이어가기

response = client.responses.create(
    model=Model,              
    input="재미있는 농담을 해줘.",  
)

# 모델이 생성한 첫 번째 응답 출력
print(response.output_text)    


# 두 번째 요청: 이전 응답(previous_response_id)을 이어받아 대화 계속
#    → 모델이 직전의 대화 맥락(response.id)을 기억하도록 전달
second_response = client.responses.create(
    model=Model,                       
    previous_response_id=response.id,   # 직전 응답 ID 지정 (대화 연결)
    input=[                             # 새로운 사용자 입력
        {"role": "user", "content": "그게 왜 재미 있는지 설명해줘"}  
    ],
)

# 모델이 이전 대화를 기반으로 한 두 번째 응답 출력
print(second_response.output_text)

좋아요! 몇 가지 간단하고 가벼운 농담 드릴게요.

- 수학 책이 왜 늘 슬프게 울까요? 문제만 계속 늘어나니까요.
- 컴퓨터가 바다에 빠지면 무슨 일이 있을까요? 웹이 끊겨서 헤엄칠 수 없으니까요.
- 빵이 공부를 좋아하는 이유는? 면학을 좋아해서요.

원하는 분위기나 주제(예: 말장난, 냉소적, 가족 친화적 등) 알려주면 더 맞춤형으로 즐거운 농담도 만들어드릴게요!
좋아요. 각 농담이 왜 재미있는지 간단히 짚어볼게요.

- 수학 책이 왜 늘 슬프게 울까요? 문제만 계속 늘어나니까요.
  - 포인트: 의인화(책이 스스로 울다)와 이중의 의미를 노린 말장난이죠. “문제”는 수학의 연습문제를 가리키지만, 삶의 문제도 떠오르게 하니까 예상과 다른 결말(책이 슬프다)이 웃음을 만듭니다.

- 컴퓨터가 바다에 빠지면 무슨 일이 있을까요? 웹이 끊겨서 헤엄칠 수 없으니까요.
  - 포인트: 의도된 오해와 단순한 결말의 대비. 바다에 빠진 컴퓨터와 인터넷 연결 상태를 연결 지어, “웹이 끊겨 헤엄칠 수 없다”는 엉뚱한 이유로 해학을 만듭니다. 말의 방향이 갑자기 바뀌는 구성이 핵심 미스디렉션입니다.

- 빵이 공부를 좋아하는 이유는? 면학을 좋아해서요.
  - 포인트: 언어유희(pun). 실제로는 “면학”이 '열심히 공부한다'는 뜻의 일반적인 단어인데, 빵과 연결되는 말로 들려서 의도되지 않은 연상을 만들어 웃습니다. 짧고 순수한 말장난이 특징이에요.

더 쉽게 말해, 이 농담들은 모두 짧고 가벼운 의인화와 단어의 이중 의미를 이용한 언어유희 humor입니다. 웃음의 핵심은 기대를 살짝 바꿔버리는 반전과, 일상적인 대상에 의인화를 더해 상황을 비현실적으로 만드는 데 있어요.

원하신다면 분위기나 주제(말장난, 냉소적, 가족 친화적 등)에 맞춘 더 많은 농담도 만들어드릴게요. 어떤 스타일을 원하나요?


### 문서/텍스트를 기반으로 챗봇이 답변하도록 하기

문서/텍스트 내용을 API 호출시 넘겨주면 우리가 원하는 답변을 유도할 수 있습니다.

In [5]:
dataset="""Document content:
질문: 인공지능 연구소의 eRecruit 시스템은 무엇입니까? 답변: 인공지능 연구소의 eRecruit 시스템은 지원자가 하나 이상의 광고된 인공지능 연구소 채용 공고에 지원서를 제출하기 위해 정기적으로 업데이트할 수 있는 개인 프로필을 만들 수 있는 온라인 시스템입니다.
질문: 인공지능 연구소의 eRecruit 시스템에 어떻게 액세스합니까? 답변: 인공지능 연구소의 eRecruit 시스템은 다음 링크를 통해 접속할 수 있습니다: https://인공지능연구소.careers.partneragency.org/erecruit.html
질문: 인공지능 연구소의 eRecruit 시스템을 사용하여 지원하는 절차는 무엇입니까? 답변: 지원 절차 지원 절차
질문: 인공지능 연구소의 eRecruit 시스템을 사용하여 온라인으로 지원해야 합니까? 답변: 모든 지원서는 인공지능 연구소의 eRecruit 시스템을 사용하여 온라인으로 제출해야 합니다. 오프라인 서면 지원서나 이메일을 통한 지원서는 접수되지 않습니다.
질문: 인공지능 연구소의 eRecruit 시스템과 호환되는 브라우저는 무엇입니까? 답변: 인공지능 연구소의 eRecruit 시스템은 Google Chrome, Internet Explorer 6 이상에 최적화되어 있습니다. 호환성 보기 모드를 사용하려면 Internet Explorer 9을 사용해야 합니다. 신청서가 성공적으로 제출되었는지 확인하려면 다음 브라우저 중 하나를 사용하는 것이 좋습니다.
질문: 내 프로필에 로그인하는 데 문제가 있습니다. 브라우저가 응답하지 않습니다. 어떻게 해야 하나요? 답변: 구인 신청을 위해 인공지능 연구소 eRecruit 프로필에 로그인하는 데 문제가 있는 경우 이는 여러 가지 이유에서 비롯될 수 있으며 그 중 일부는 인터넷 연결과 같은 인공지능 연구소의 통제 범위를 벗어납니다. 그러나 이러한 유형의 문제를 해결하려면 다음 지침을 따르는 것이 좋습니다. • 권장 브라우저(및 버전)를 사용하고 있는지 확인하십시오. • 캐시/브라우저 기록을 지웁니다. http://www.refreshyourcache.com/en/home에서 브라우저 기록을 지우는 방법에 대한 정보를 찾을 수 있습니다. 이 작업을 수행하기 전에 브라우저에서 캐시를 지울 때의 결과를 알고 있는지 확인하십시오.
질문: 인공지능 연구소의 eRecruit 시스템을 사용하는 데 도움이 필요하면 어떻게 합니까? 답변: 인공지능 연구소의 eRecruit 시스템 사용에 대한 일반적인 질문이나 지원이 필요한 경우 헬프데스크(https://info.인공지능연구소.org/sas/erecruit/Assets/HelpDesk.aspx)에 문의하세요.
질문: 왜 등록해야 합니까? 답변: 모든 지원자는 먼저 인공지능 연구소의 eRecruit 시스템에 등록해야 합니다. 등록이 완료되면 개인 정보를 입력하고 광고된 채용 공고에 지원할 수 있도록 개인 계정이 생성됩니다.
질문: 등록할 때 사용자 이름으로 무엇을 사용해야 합니까? 답변: 인공지능 연구소 eRecruit 시스템에 등록할 때 유효한 이메일 주소를 사용자 이름으로 사용하는 것이 좋습니다.
질문: 어떤 비밀번호 형식이 허용되나요? 답변: 인공지능 연구소는 강력한 비밀번호 사용을 권장합니다. 비밀번호는 최소 8자 이상이어야 하며 문자와 숫자를 조합해야 합니다.
질문: 비밀번호를 어떻게 변경할 수 있나요? 답변: 시스템에 로그인한 후 '개인 정보' 링크에서 '비밀번호 변경' 옵션을 선택하세요. '사용자 이름 또는 비밀번호를 잊으셨나요?' 링크를 클릭하고 지침에 따라 잊어버린 비밀번호를 검색하세요.
질문: 비밀번호를 잊어버렸습니다. 어떻게 해야 하나요? 답변: 비밀번호를 잊어버린 경우, 사용자 ID 또는 비밀번호 찾기 링크를 클릭하고 두 가지 옵션 중 하나를 완료하세요.
질문: 내 데이터는 안전합니까? 답변: 인공지능 연구소의 eRecruit 데이터는 개인 보안 데이터베이스에 저장되며 인공지능 연구소는 데이터의 소유자입니다. 이 시스템은 인공지능 연구소의 엄격한 보안 요구 사항을 충족합니다.
질문: 각 섹션의 모든 정보를 작성해야 합니까? 답변: 별표(*)가 표시된 모든 항목은 필수 정보이며 각 섹션별로 작성해야 합니다.
질문: 한 세션에서 모든 정보를 완료해야 합니까? 답변: 지원 과정 중 언제든지 지원서를 저장하고 나중에 계속할 수 있습니다. 섹션을 완료할 때 정보가 손실되지 않도록 정기적으로 저장 버튼을 사용하는 것이 좋습니다.
질문: 달력 기능을 사용하여 날짜를 어떻게 선택합니까? 답변: 날짜는 두 가지 방법으로 선택할 수 있습니다. A) dd/mm/yyyy 형식을 사용하여 필드에 날짜를 직접 입력하거나 B) 달력 아이콘을 클릭하여 날짜를 선택하고 연도, 월 및 일을 선택할 수 있습니다.
질문: 관련 정보가 드롭다운 옵션에 포함되어 있지 않으면 어떻게 해야 합니까? 답변: 사용 가능한 드롭다운 옵션 중에서 선택해야 합니다. 귀하의 개인 정보, 기술 및 경험에 가장 가까운 옵션을 선택하십시오.
질문: 조회 기능을 어떻게 사용합니까? 답변: 조회 기능을 사용하려면 돋보기 아이콘을 클릭하세요. 그런 다음 해당 필드에 전체 또는 부분 값을 입력하고 조회 버튼을 클릭합니다. 마지막으로 검색 결과에서 적절한 값을 선택하세요.
질문: 맞춤법 검사 기능을 사용할 수 있나요? 답변: 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 클릭 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 사전을 클릭하세요.
"""

In [6]:
# OpenAI Responses API 예제: 콜센터 도우미 시뮬레이션

response = client.responses.create(
    model=Model, 

    # 모델의 역할을 정의하는 system-level 지침
    instructions="""
        콜센터 응답자 역할을 해주시기 바랍니다.
        당신의 이름은 "콜센터 도우미"입니다.

        첫 인사 규칙:
        처음 대화 시작 시에는 반드시
        '안녕하세요 고객님. 무엇을 도와드릴까요?' 라고 말하세요.

        응답 규칙:
        - 당신은 제공된 정보(dataset)에 대한 질문에만 답변해야 합니다.
        - 데이터에 해당 답변이 없으면 '죄송합니다. 정보가 없습니다.'라고 말하고 대화를 중지하세요.
        - 정보와 무관한 질문에는 반드시 답변을 거부하세요.
    """,

    # 대화의 시작점 (사용자 입력)
    input=[{"role": "user", "content": dataset}]
)

# 모델의 첫 번째 응답(초기 인사 또는 안내문) 출력
print(response.output_text)

# 메인 대화 루프: 사용자가 'quit' 입력 시까지 계속 대화 유지
while True:
    user_input = input("사용자 입력: ")

    # 사용자가 'quit'를 입력하면 루프 종료
    if user_input.lower() == "quit":
        break

    # 모델에게 이전 대화 맥락(previous_response_id)을 전달하여 대화 지속
    response = client.responses.create(
        model=Model,
        previous_response_id=response.id,  # 이전 응답 ID를 전달하여 대화 상태 유지
        input=[{"role": "user", "content": user_input}],  # 새 사용자 메시지 전달
    )

    # 모델의 응답 출력
    print("assistant:", response.output_text)

안녕하세요 고객님. 무엇을 도와드릴까요?

인공지능 연구소의 eRecruit 시스템에 관한 질문이 있으시면, 제공된 데이터셋에 기반해 답변해 드리겠습니다. 질문이 있다면 편하게 물어봐 주세요.


사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?


assistant: 네. 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 클릭 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 사전을 클릭하세요.


사용자 입력:  QUIT


------------------
(대화 예시)
```
안녕하세요 고객님. 무엇을 도와드릴까요?
사용자 입력:  맞춤법 검사 기능을 사용할 수 있나요?
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 각 섹션의 다양한 텍스트 설명 필드에 대해 맞춤법 검사 기능을 사용할 수 있습니다. 맞춤법 검사 기능을 사용하려면 해당 필드 내에서 사전 클릭 옵션을 선택하시면 됩니다.
사용자 입력:  인공지능 연구소는 어떤 비밀번호 형식을 요구하나요 
assistant: 인공지능 연구소는 강력한 비밀번호 사용을 권장하며, 비밀번호는 최소 8자 이상이어야 하고 문자와 숫자를 조합해야 합니다.
사용자 입력:  각 섹션의 모든 정보를 제공해야하나요
assistant: 네, 인공지능 연구소의 eRecruit 시스템에서는 별표(*)로 표시된 모든 항목이 필수 정보로 지정되어 있으므로, 각 섹션의 모든 필수 항목을 반드시 작성해야 합니다.
사용자 입력:  강남에서 남산에 가는 교통편을 알려주세요 
assistant: 강남에서 남산으로 가는 교통편은 여러 가지가 있습니다. 대표적인 방법은 다음과 같습니다:

1. **지하철 이용**:
   - 강남역에서 2호선을 타고 '역삼역' 또는 '선릉역'에서 3호선으로 환승 → 충무로역 또는 명동역에서 하차 → 남산 방향으로 도보 혹은 버스 이용
   - 또는 강남역에서 9호선으로 환승해 '명동역' 또는 '남산촌차고지'로 이동하는 방법도 있습니다.

2. **버스 이용**:
   - 강남 일대에서 남산 순환버스(남산순환버스 또는 시내버스)를 이용할 수 있으며, 일부 버스는 남산 근처 정류장에 정차합니다.

3. **택시 또는 차량 이용**:
   - 택시를 타거나 개인 차량으로 이동하면 약 20~30분 정도 소요됩니다. 남산공원 및 케이블카 하차장 근처에 주차하거나, 케이블카를 이용하려면 남산케이블카 승강장 부근으로 이동하시면 됩니다.

시간과 교통 상황에 따라 최적의 경로를 선택하실 수 있으니, 출발 전에 교통정보 앱이나 지도 서비스를 확인하시는 것을 추천드립니다.
사용자 입력:  quit
```

## 대화 상태를 위한 OpenAI API

OpenAI의 API를 사용하면 각 대화(turn)마다 입력을 수동으로 전달하지 않아도, 대화 상태(conversation state) 를 자동으로 관리할 수 있습니다.

### Conversations API 

Conversations API 는 Responses API 와 함께 작동하여,
지속 가능한 ID를 가진 장기 실행 객체(long-running object) 로서 대화 상태를 유지합니다.
한 번 생성된 대화 객체는 세션, 기기, 작업(job) 간에 계속 사용할 수 있습니다.

Conversations는 메시지, 도구 호출(tool calls), 도구 출력(tool outputs), 기타 데이터 등의 항목(items) 을 저장합니다. 

여러 턴의 대화에서, 이전 응답을 일일이 연결할 필요 없이
이 conversation 객체를 이후 요청에 전달하면 대화 상태와 컨텍스트를 지속적으로 공유할 수 있습니다.

In [7]:
import openai

conversation = openai.conversations.create()
conversation_id = conversation.id
conversation_id

'conv_690eb10cd5408194a2dda61a917c67c80e5f09a48deff1da'

In [9]:
response = openai.responses.create(
  model=Model,
  input=[{"role": "user", "content": "이순신 장군은 몇번을 싸워 이겼나요?"}],
  conversation=conversation_id
)

print(response.output_text)

일반적으로 전해지는 이야기로는, 이순신 장군은 임진왜란 기간에 해전에서 23번의 전투를 벌여 모두 이겼다고 전해져요. 즉, 23전 23승으로 많이 알려져 있습니다. 다만 기록에 따라 전투 수를 다르게 집계하는 경우가 있어 숫자에 차이가 있을 수 있습니다.

대표적인 승리 전투로는 옥포해전, 사천해전, 한산도 대첩, 명량해전, 노량해전 등이 있습니다. 필요하시면 각 전투의 간단한 요약도 드릴게요.


In [10]:
response = openai.responses.create(
  model=Model,
  input=[{"role": "user", "content": "그 중에 마지막 전투는 어느 전투인가요?"}],
  conversation=conversation_id
)

print(response.output_text)

마지막 전투는 노량해전(1598년)입니다. 이 전투에서 이순신 장군이 전사했고, 임진왜란의 해상전 가운데 마지막 전투로 간주됩니다.
